In [1]:
import pandas as pd
import requests
import os
from pathlib import Path
import os.path
import statsmodels
from statsmodels.formula.api import *
import matplotlib.pyplot as plt
from sklearn import linear_model
from ggplot import *
from scipy import stats

In [2]:
reviews_df = pd.read_csv("PT_review_data.csv")
reviews_df

,name,slug,type,course,rating,review,date
0,A Anthony,anthony,professor,AMST203,1.0,"By far the worst professor I’ve ever had, and ...",2018-08-17
1,A Kruglanski,kruglanski,professor,PSYC489H,2.0,"DO NOT TAKE PSYC489H ""Motivated Social Cogniti...",2015-09-07
2,A Sharma,sharma_a,professor,ASTR300,2.0,"Very boring, it's hard to maintain your focus ...",2019-04-04
3,A Sharma,sharma_a,professor,ASTR300,1.0,You'll pass but this class will be the most bo...,2019-05-26
4,A Sharma,sharma_a,professor,ASTR300,1.0,Rather difficult course. Class is extremely bo...,2019-12-08
...,...,...,...,...,...,...,...
19180,Zsuzsa Daczo,daczo,professor,SOCY105,5.0,Amazing professor. She genuinely cares about h...,2020-09-01
19181,Zsuzsa Daczo,daczo,professor,SOCY105,3.0,I had Zsuzsa for SOCY105 online. Lectures were...,2021-01-12
19182,Zsuzsa Daczo,daczo,professor,SOCY105,5.0,"Thoughtful, kind, and really funny. She talks ...",2021-01-22
19183,Zubin Jelveh,jelveh,professor,CCJS418E,1.0,Possibly the most clueless professor regarding...,2022-04-07


In [3]:
# assume that all reviews from sep to dec are for fall classes, jan are for winter classes,
# jun and jul are for summer classes, and feb to may are for spring classes

for index, row in reviews_df.iterrows():
    reviews_df.at[index, "year"] = str(reviews_df.at[index, "date"])[:4]
    if int(str(reviews_df.at[index, "date"])[5:7]) > 8:
        reviews_df.at[index, "season"] = "fall"
        #fallcount = fallcount + 1
    elif str(reviews_df.at[index, "date"])[5:7] == "01":
        reviews_df.at[index, "season"] = "winter"
        #wincount = wincount+1
    elif str(reviews_df.at[index, "date"])[5:7] == "06" or str(reviews_df.at[index, "date"])[5:7] == "07":
        reviews_df.at[index, "season"] = "summer"
    else:
        reviews_df.at[index, "season"] = "spring"
        #springcount = springcount + 1

reviews_df

,name,slug,type,course,rating,review,date,year,season
0,A Anthony,anthony,professor,AMST203,1.0,"By far the worst professor I’ve ever had, and ...",2018-08-17,2018,spring
1,A Kruglanski,kruglanski,professor,PSYC489H,2.0,"DO NOT TAKE PSYC489H ""Motivated Social Cogniti...",2015-09-07,2015,fall
2,A Sharma,sharma_a,professor,ASTR300,2.0,"Very boring, it's hard to maintain your focus ...",2019-04-04,2019,spring
3,A Sharma,sharma_a,professor,ASTR300,1.0,You'll pass but this class will be the most bo...,2019-05-26,2019,spring
4,A Sharma,sharma_a,professor,ASTR300,1.0,Rather difficult course. Class is extremely bo...,2019-12-08,2019,fall
...,...,...,...,...,...,...,...,...,...
19180,Zsuzsa Daczo,daczo,professor,SOCY105,5.0,Amazing professor. She genuinely cares about h...,2020-09-01,2020,fall
19181,Zsuzsa Daczo,daczo,professor,SOCY105,3.0,I had Zsuzsa for SOCY105 online. Lectures were...,2021-01-12,2021,winter
19182,Zsuzsa Daczo,daczo,professor,SOCY105,5.0,"Thoughtful, kind, and really funny. She talks ...",2021-01-22,2021,winter
19183,Zubin Jelveh,jelveh,professor,CCJS418E,1.0,Possibly the most clueless professor regarding...,2022-04-07,2022,spring


In [4]:
reviews_df["season"].value_counts()

season
fall      9137
spring    6986
winter    2080
summer     982
Name: count, dtype: int64

In [5]:
reviews_df = reviews_df.sort_values(by = ["name", "course", "year", "season"], ignore_index = True)

average_reviews_df = pd.DataFrame()

values_to_check = ["name", "course", "year", "season"]
curr = [reviews_df.at[0, "name"], reviews_df.at[0, "course"], reviews_df.at[0, "year"], reviews_df.at[0, "season"]]
count = 0
averages = [0, 0]
for index, row in reviews_df.iterrows():
    match = True
    for i in range(4):
        if curr[i] != reviews_df.at[index, values_to_check[i]]:
            match = False
    if not match:
        for i in range(4):
            average_reviews_df.at[count, values_to_check[i]] = curr[i]
            curr[i] = reviews_df.at[index, values_to_check[i]]
        average_reviews_df.at[count, "average_rating"] = averages[0]/averages[1]
        average_reviews_df.at[count, "num_reviews"] = averages[1]
        count = count + 1
        averages = [0, 0]
    averages[0] = averages[0] + reviews_df.at[index, "rating"]
    averages[1] = averages[1] + 1

average_reviews_df

,name,course,year,season,average_rating,num_reviews
0,A Anthony,AMST203,2018,spring,1.0,1.0
1,A Kruglanski,PSYC489H,2015,fall,2.0,1.0
2,A Sharma,ASTR300,2019,fall,1.0,1.0
3,A Sharma,ASTR300,2019,spring,1.5,2.0
4,A Sharma,ASTR300,2020,fall,2.0,1.0
...,...,...,...,...,...,...
12766,Zohreh Davoudi,PHYS411,2022,spring,1.5,2.0
12767,Zsuzsa Daczo,SOCY105,2019,spring,5.0,1.0
12768,Zsuzsa Daczo,SOCY105,2020,fall,5.0,1.0
12769,Zsuzsa Daczo,SOCY105,2021,winter,4.0,2.0


In [6]:
grades_df = pd.read_csv("PT_grade_data.csv")
grades_df

,professor,course,semester,section,A+,A,A-,B+,B,B-,C+,C,C-,D+,D,D-,F,W,Other
0,A Anthony,AMST202,201608,0101,1.0,2.0,1.0,2.0,5.0,1.0,2.0,3.0,5.0,0.0,0.0,0.0,2.0,4.0,0.0
1,A Anthony,AMST202,201701,0101,0.0,1.0,4.0,3.0,3.0,7.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,A Anthony,AMST203,201708,FCH1,0.0,2.0,1.0,1.0,6.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
3,A Anthony,AMST203,201708,FCH2,0.0,0.0,1.0,4.0,3.0,0.0,0.0,4.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0
4,A Anthony,AMST203,201801,0201,1.0,3.0,1.0,2.0,4.0,0.0,2.0,3.0,1.0,0.0,0.0,2.0,2.0,6.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87241,Zubin Jelveh,CCJS418E,202201,0101,22.0,4.0,3.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0
87242,Zubin Jelveh,CCJS418E,202208,101,1.0,1.0,9.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0
87243,Zubin Jelveh,INST414,202101,0103,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
87244,Zubin Jelveh,INST414,202108,0103,36.0,1.0,2.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,5.0,0.0


In [7]:
for index, row in grades_df.iterrows():
    grades_df.at[index, "year"] = str(grades_df.at[index, "semester"])[:4]
    if str(grades_df.at[index, "semester"])[4:] == "08":
        grades_df.at[index, "season"] = "fall"
    else:
        grades_df.at[index, "season"] = "spring"

grades_df

,professor,course,semester,section,A+,A,A-,B+,B,B-,...,C,C-,D+,D,D-,F,W,Other,year,season
0,A Anthony,AMST202,201608,0101,1.0,2.0,1.0,2.0,5.0,1.0,...,3.0,5.0,0.0,0.0,0.0,2.0,4.0,0.0,2016,fall
1,A Anthony,AMST202,201701,0101,0.0,1.0,4.0,3.0,3.0,7.0,...,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2017,spring
2,A Anthony,AMST203,201708,FCH1,0.0,2.0,1.0,1.0,6.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,2017,fall
3,A Anthony,AMST203,201708,FCH2,0.0,0.0,1.0,4.0,3.0,0.0,...,4.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,2017,fall
4,A Anthony,AMST203,201801,0201,1.0,3.0,1.0,2.0,4.0,0.0,...,3.0,1.0,0.0,0.0,2.0,2.0,6.0,0.0,2018,spring
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87241,Zubin Jelveh,CCJS418E,202201,0101,22.0,4.0,3.0,2.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,2022,spring
87242,Zubin Jelveh,CCJS418E,202208,101,1.0,1.0,9.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,2022,fall
87243,Zubin Jelveh,INST414,202101,0103,2.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2021,spring
87244,Zubin Jelveh,INST414,202108,0103,36.0,1.0,2.0,1.0,0.0,2.0,...,0.0,0.0,0.0,0.0,1.0,1.0,5.0,0.0,2021,fall


In [8]:
# takes a long time, ~ 1.5min, can uncomment print(index) to track progress if you want

grades_df = grades_df.sort_values(by = ["professor", "course", "year", "season"], ignore_index = True)

average_grades_df = pd.DataFrame()
gpa = {"A+":4.0, "A":4.0, "A-":3.7,"B+":3.3, "B":3.0, "B-":2.7, "C+":2.3, "C":2.0, "C-":1.7, "D+":1.3, "D":1.0, "D-":0.7, "F":0.0}

values_to_check = ["professor", "course", "year", "season"]
curr = [grades_df.at[0, "professor"], grades_df.at[0, "course"], grades_df.at[0, "year"], grades_df.at[0, "season"]]
count = 0
averages = [0, 0]
for index, row in grades_df.iterrows():
    #print(index)
    match = True
    for i in range(4):
        if curr[i] != grades_df.at[index, values_to_check[i]]:
            match = False
    if not match:
        for i in range(4):
            if averages[1] != 0:
                average_grades_df.at[count, values_to_check[i]] = curr[i]
            curr[i] = grades_df.at[index, values_to_check[i]]
        if averages[1] != 0:
            average_grades_df.at[count, "average_gpa"] = averages[0]/averages[1]
            average_grades_df.at[count, "num_students"] = averages[1]
            count = count + 1
        averages = [0, 0]
    for i in gpa.keys():
        averages[0] = averages[0] + gpa.get(i)*grades_df.at[index, i]
        averages[1] = averages[1] + grades_df.at[index, i]

average_grades_df

,professor,course,year,season,average_gpa,num_students
0,A Anthony,AMST202,2016,fall,2.462500,24.0
1,A Anthony,AMST202,2017,spring,2.934783,23.0
2,A Anthony,AMST203,2017,fall,2.796429,28.0
3,A Anthony,AMST203,2018,fall,3.204651,43.0
4,A Anthony,AMST203,2018,spring,2.476190,21.0
...,...,...,...,...,...,...
47775,Zubin Jelveh,CCJS418E,2021,fall,3.624000,25.0
47776,Zubin Jelveh,CCJS418E,2022,fall,3.328571,14.0
47777,Zubin Jelveh,CCJS418E,2022,spring,3.778788,33.0
47778,Zubin Jelveh,INST414,2021,fall,3.745455,44.0


In [9]:
average_grades_df = average_grades_df.rename(columns = {"professor" : "name"})

both_averages_df = pd.merge(
    average_reviews_df,
    average_grades_df,
    how = "outer",
    on = ["name", "course", "year", "season"]
)

In [10]:
import numpy as np

def combine(group):
     return pd.Series({
        'course': group['course'].tolist(),
        'semester': [a + " " + b for a, b in zip(group['season'], group['year'])],
        'average_rating': group['average_rating'].tolist(),
        'num_reviews': group['num_reviews'].tolist(),
        'average_gpa': [round(x, 2) for x in group['average_gpa']],
        'num_students': [int(x) for x in np.nan_to_num(group['num_students'])]
    })
df_grouped = both_averages_df.groupby(['name']).apply(combine).reset_index()
df_grouped["name"] = df_grouped["name"].apply(lambda x: x.split()[0].upper()) + " " +df_grouped["name"].apply(lambda x: x.split()[-1].upper())


#print(df_grouped.to_string())
duplicates = df_grouped[df_grouped.duplicated(['name'], keep=False)]

# get first two rows
result = duplicates.head(100)

print(result)

                name                                 course   
563           DA LEE                   [ENGL101X, ENGL101X]  \
564           DA LEE   [AAST233, AAST233, CMLT270, CMLT275]   
1417  JOSE CIFUENTES  [MATH136, MATH140, MATH141H, STAT410]   
1422  JOSE CIFUENTES                              [MATH140]   

                                              semester        average_rating   
563                           [spring 2020, fall 2019]            [5.0, nan]  \
564     [fall 2022, fall 2021, fall 2022, spring 2022]  [5.0, nan, nan, nan]   
1417  [fall 2022, spring 2022, spring 2022, fall 2022]  [5.0, nan, nan, nan]   
1422                                     [spring 2023]                 [5.0]   

               num_reviews               average_gpa      num_students  
563             [1.0, nan]               [nan, 2.96]           [0, 12]  
564   [1.0, nan, nan, nan]     [nan, 3.74, 3.8, 3.5]   [0, 29, 25, 27]  
1417  [1.0, nan, nan, nan]  [3.26, 2.08, 2.78, 2.81]  [10, 213, 

In [11]:
df_grouped.to_csv(f'PT_combined_data.csv', index=False)